# Рекурсия с числами

**Рекурсия** — это способ определения функции, при котором функция вызывает сама себя на более простой версии задачи.

Любая рекурсивная функция состоит из двух обязательных частей:
1. Базовый случай — самый простой вариант задачи, который решается напрямую, без рекурсивного вызова.
2. Рекурсивный случай — переход от текущей задачи к более простой (обычно аргумент уменьшается или структура «сжимается»).

Если базового случая нет или его условие никогда не выполнится — рекурсия зациклится и уйдет в бесконечность (а потом программа упадет)

При рекурсии работает очередь вызовов:
- каждый вызов функции кладётся в очередь;
- когда достигаем базового случая, результаты начинают возвращаться назад — очередь "разворачивается"

Самый простой пример рекурсии - факториал. В терминах рекурсии
- `0! = 1` — базовый случай
- `n! = n*(n - 1)!` при `n>0` — рекурсивный случай

In [1]:
def fact(n):
    if n == 0:
        return 1
    return n * fact(n - 1)

print(fact(5))

120


Теперь посмотрим на более "затратную" версию рекурсии - числа Фибоначчи. Они задаются рекурсивно (следующее число зависит от двух предыдущих):
- `F(0) = 1, F(1) = 1` — базовые случаи
- `F(n) = F(n - 1) + F(n - 2)` — рекурсивный случай

In [3]:
def fib(n):
    if n == 0 or n == 1:
        return 1
    return fib(n - 1) + fib(n - 2)

print(fib(8))

34


Здесь есть два рекурсивных вызова, поэтому количество вычислений растёт экспоненциально. Мы многократно будем пересчитывать одни и те же значения.

Есть менее известные числа Леонардо. Формула похожа на числа Фибоначчи, только добавляется `+1`

In [4]:
def leo(n):
    if n == 0 or n == 1:
        return 1
    return leo(n - 1) + leo(n - 2) + 1

print(leo(8))

67


С числами Леонардо возникает такая же проблема, что и с Фибоначчи - экспоненциальный рост и большие расходы по памяти и времени

Строго говоря, любую рекуррентную формулу можно почти буквально переписать в виде рекурсивной функции (что мы и сделали)

# Рекурсия со строками

Рассмотрим такой шаблон:
- есть "алфавит" возможных символов
- есть текущая построенная строка
- пытаемся сгенерировать новые строки
- если строка уже готова, то выводим её, а если нет - пытаемся добавить очередной символ и рекурсивно продолжаем

## Перестановки без повторений

Нужно вывести все возможные перестановки строки, причем так, чтобы каждый символ использовался ровно один раз

Давайте на каждом шаге выбирать один символ из тех, что остались (`head`), добавлять его к `tail` и рекурсивно переставлять остаток

In [6]:
def gena(head, tail):
  if not head:  # базовый случай - если символов не осталось - строка готова, можно выходить из рекурсии
    print(tail)
  else:
    for i in range(len(head)):
      gena(head[:i] + head[i+1:], tail + head[i])

gena('abc', '')

abc
acb
bac
bca
cab
cba


## Строки фиксированной длины с повторениями

Задача похожа на предыдущую, но теперь мы можем использовать символ несколько раз. Поэтому, мы не будем удалять выбранный символ из `head`

In [ ]:
def gena(head,tail):
  if len(head) == len(tail):  # базовый случай - как только получили строку нужной длины - выходим
    print(tail)
  else:
    for i in range(len(head)):
      gena(head, tail + head[i])

gena('abc', '')

aaa
aab
aac
aba
abb
abc
aca
acb
acc
baa
bab
bac
bba
bbb
bbc
bca
bcb
bcc
caa
cab
cac
cba
cbb
cbc
cca
ccb
ccc


## Строки длины $n$ из заданного алфавита

Здесь мы еще на один шаг абстрагируемся от изначального условия - теперь нам нужно составить все возможные строки заданной длины из заданных символов (алфавита)

В предыдущем примере длина сгенерированной строки определялась количеством символов в алфавите

In [8]:
n = 3
def gena(head,tail):
  if len(tail) == n:
    print(tail)
  else:
    for i in range(len(head)):
      gena(head, tail + head[i])

gena('012', '')

000
001
002
010
011
012
020
021
022
100
101
102
110
111
112
120
121
122
200
201
202
210
211
212
220
221
222


## Перестановки с маской

Дана строка `st` - набор символов, которые будут использоваться при генерации строк. В отличие от предыдущих примеров мы не будем изменять эту строку.

Идея заключается в том, что мы будем хранить текущий результат `ans`, то есть уже собранную строку. А также будем поддерживать список `mask` длины 3, в котором `mask[i] = 0` означает, что `i`-й символ ещё не использован.

На каждом шаге алгоритма мы:
- перебираем все возможные символы, которые ещё не использованы
- берём один неиспользованный символ и кладём его в `ans`
- отмечаем, что он использован, то есть `mask[i] = 1`
- рекурсивно вызываем генерацию строкидля оставшихся неиспользованных символов
- после возврата из рекурсии откатываем изменения, то есть `mask[i] = 0` и `ans.pop()`

In [9]:
st = 'abc'
ans = []
mask = [0 for i in range(len(st))]

def gena():
  if len(ans) == len(st): # как только собрали перестановку - выходим из рекурсии
    print(*ans, sep='')
  else:
    for i in range(len(mask)):  # перебираем индексы 0,1,2
      if mask[i] < 1: # если mask[i] = 0, то символ st[i] можно использовать
        mask[i] += 1
        ans.append(st[i])
        gena()
        mask[i] -= 1
        ans.pop()

gena()

abc
acb
bac
bca
cab
cba
